In [16]:
import json
from pprint import pprint

In [17]:
with open('train.json') as f:
    data1=json.load(f)

data1.keys()
#pprint(data)

dict_keys(['index', 'content', 'date', 'tags', 'title', 'url'])

In [18]:
with open('test.json') as f:
    data2=json.load(f)

data1.keys()
#pprint(data)

dict_keys(['index', 'content', 'date', 'tags', 'title', 'url'])

In [36]:
import pandas as pd
set=[]
for k,v in data1['content'].items():
    #print(k)
    #print(v)
    set.append(v)
    
import pandas as pd

for k,v in data2['content'].items():
    #print(k)
    #print(v)
    set.append(v)
    
df1=pd.DataFrame(set)
print(df1.head())
print(df1.shape)

                                                   0
0  Honda is recalling the 2017 Honda Ridgeline pi...
1  This morning, Honda issued two safety recalls ...
2  2016 was dominated by some pretty major headli...
3  Last week, Hyundai recalled some 5,600 crossov...
4  Fiat Chrysler Automobiles is recalling nearly ...
(2835, 1)


In [37]:
df1.columns=['a']

In [38]:
df1['a']

0       Honda is recalling the 2017 Honda Ridgeline pi...
1       This morning, Honda issued two safety recalls ...
2       2016 was dominated by some pretty major headli...
3       Last week, Hyundai recalled some 5,600 crossov...
4       Fiat Chrysler Automobiles is recalling nearly ...
5       Hyundai is recalling more than 5,600 Tucson an...
6       The massive, record-breaking recall of Takatas...
7       The National Highway Traffic Safety Administra...
8       A little over a year ago, AutoNation made a ve...
9       Ford has issued two recalls affecting nearly 6...
10      Nissan is recalling select 2016 Titan Diesel X...
11      Toyota is recalling the 2011-2016 Toyota Sienn...
12      Kia has announced a recall of the 2008 and 200...
13      This morning, Fiat Chrysler Automobiles issued...
14      Mazda is recalling RX-8 vehicles from the 2004...
15      Mitsubishi is recalling roughly 195,000 U.S. v...
16      Volvo has issued recalls for 2016 and 2017 mod...
17      Kia is

In [39]:
#nltk.download()

In [40]:

import nltk
from nltk.corpus import stopwords
mystopwords=stopwords.words("English") + ['one', 'become', 'get', 'make', 'take', 'recall','said', 'say', 'could', 'nhtsa', 'n\'t', 'may', 'vehicle', 'car']
WNlemma = nltk.WordNetLemmatizer()

def pre_process(text):
    tokens = nltk.word_tokenize(text)
    tokens=[ WNlemma.lemmatize(t.lower()) for t in tokens]
    tokens=[ t for t in tokens if t not in mystopwords]
    tokens = [ t for t in tokens if len(t) >= 3 ]
    return(tokens)

In [41]:
toks_train = df1['a'].apply(pre_process)

In [42]:
#!pip install gensim

In [43]:
import logging
import gensim 
from gensim import corpora

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [44]:
# Prepare a vocabulary dictionary.
dictionary = corpora.Dictionary(toks_train)
print(dictionary)

2018-10-10 17:35:38,656 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2018-10-10 17:35:39,178 : INFO : built Dictionary(28216 unique tokens: ['1-888-234-2138', '1-888-327-4236', '16v888000', '2016', '2017']...) from 2835 documents (total 544361 corpus positions)


Dictionary(28216 unique tokens: ['1-888-234-2138', '1-888-327-4236', '16v888000', '2016', '2017']...)


In [45]:
type(dictionary)

gensim.corpora.dictionary.Dictionary

In [46]:
dtm_train = [dictionary.doc2bow(d) for d in toks_train ]

In [47]:
# Build an LDA model for 3 topics out of the DTM
%time lda = gensim.models.ldamodel.LdaModel(dtm_train, num_topics = 5,  alpha='auto',id2word = dictionary)
#chunksize (int, optional) – Number of documents to be used in each training chunk.
#passes (int, optional) – Number of passes through the corpus during training.
#chunksize=30,
#passes = 20

2018-10-10 17:35:39,945 : INFO : using autotuned alpha, starting with [0.2, 0.2, 0.2, 0.2, 0.2]
2018-10-10 17:35:39,947 : INFO : using symmetric eta at 0.2
2018-10-10 17:35:39,953 : INFO : using serial LDA version on this node
2018-10-10 17:35:39,978 : INFO : running online (single-pass) LDA training, 5 topics, 1 passes over the supplied corpus of 2835 documents, updating model once every 2000 documents, evaluating perplexity every 2835 documents, iterating 50x with a convergence threshold of 0.001000
2018-10-10 17:35:39,979 : WARNING : too few updates, training might not converge; consider increasing the number of passes or iterations to improve accuracy
2018-10-10 17:35:39,980 : INFO : PROGRESS: pass 0, at document #2000/2835
2018-10-10 17:35:42,529 : INFO : optimized alpha [0.3057041, 0.31045505, 0.34456867, 0.28837162, 0.34303892]
2018-10-10 17:35:42,532 : INFO : merging changes from 2000 documents into a model of 2835 documents
2018-10-10 17:35:42,566 : INFO : topic #0 (0.306): 0.

Wall time: 5.14 s


In [48]:
# To see the topics, with the most probable words in each topic. What topics to you see? 
lda.show_topics()

[(0,
  '0.009*"safety" + 0.007*"owner" + 0.007*"year" + 0.007*"affected" + 0.006*"customer" + 0.006*"company" + 0.006*"number" + 0.005*"dealer" + 0.005*"model" + 0.005*"motor"'),
 (1,
  '0.009*"safety" + 0.008*"owner" + 0.007*"dealer" + 0.006*"seat" + 0.005*"model" + 0.005*"issue" + 0.005*"ford" + 0.005*"volkswagen" + 0.004*"automaker" + 0.004*"bmw"'),
 (2,
  '0.016*"owner" + 0.011*"dealer" + 0.010*"affected" + 0.010*"safety" + 0.009*"ford" + 0.008*"model" + 0.008*"service" + 0.006*"inflator" + 0.006*"problem" + 0.006*"2015"'),
 (3,
  '0.007*"safety" + 0.007*"chrysler" + 0.007*"company" + 0.006*"fca" + 0.006*"2015" + 0.005*"owner" + 0.005*"takata" + 0.005*"customer" + 0.005*"problem" + 0.005*"ford"'),
 (4,
  '0.012*"model" + 0.010*"safety" + 0.009*"owner" + 0.009*"affected" + 0.008*"year" + 0.007*"number" + 0.006*"problem" + 0.006*"2014" + 0.006*"toyota" + 0.005*"dealer"')]

In [49]:
lda.show_topics(num_words=10)

[(0,
  '0.009*"safety" + 0.007*"owner" + 0.007*"year" + 0.007*"affected" + 0.006*"customer" + 0.006*"company" + 0.006*"number" + 0.005*"dealer" + 0.005*"model" + 0.005*"motor"'),
 (1,
  '0.009*"safety" + 0.008*"owner" + 0.007*"dealer" + 0.006*"seat" + 0.005*"model" + 0.005*"issue" + 0.005*"ford" + 0.005*"volkswagen" + 0.004*"automaker" + 0.004*"bmw"'),
 (2,
  '0.016*"owner" + 0.011*"dealer" + 0.010*"affected" + 0.010*"safety" + 0.009*"ford" + 0.008*"model" + 0.008*"service" + 0.006*"inflator" + 0.006*"problem" + 0.006*"2015"'),
 (3,
  '0.007*"safety" + 0.007*"chrysler" + 0.007*"company" + 0.006*"fca" + 0.006*"2015" + 0.005*"owner" + 0.005*"takata" + 0.005*"customer" + 0.005*"problem" + 0.005*"ford"'),
 (4,
  '0.012*"model" + 0.010*"safety" + 0.009*"owner" + 0.009*"affected" + 0.008*"year" + 0.007*"number" + 0.006*"problem" + 0.006*"2014" + 0.006*"toyota" + 0.005*"dealer"')]